# Notes: 
## This notebook is pretty much based on [this](https://www.kaggle.com/alekseyromanovich/leaked-data)
## I'm pretty new to Kaggle community. I couldn't deny possible violation to the rules. Let me know if it's the case

In [ ]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
!pip install openpyxl

## Data loading and processing

In [ ]:
data_leaked = pd.read_excel('/kaggle/input/air-quality-time-series-data-uci/AirQualityUCI.xlsx')
test = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv')
train = pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/train.csv')

In [ ]:
train[:5]

In [ ]:
test

In [ ]:
co = train["target_carbon_monoxide"]
benzene = train["target_benzene"]
nox = train["target_nitrogen_oxides"]
train = train.drop(["date_time","target_carbon_monoxide", "target_benzene", "target_nitrogen_oxides"], axis = 1)

In [ ]:
test = test.drop("date_time", axis = 1)

In [ ]:
train

In [ ]:
test

## Linear regression 

In [ ]:
from sklearn.linear_model import LinearRegression
lr_co = LinearRegression()
lr_benzene = LinearRegression()
lr_nox = LinearRegression()

In [ ]:
lr_co.fit(train, co)
lr_benzene.fit(train, benzene)
lr_nox.fit(train, nox)

In [ ]:
co_pred = lr_co.predict(test)
benzene_pred = lr_benzene.predict(test)
nox_pred = lr_nox.predict(test)

## Fitting leaked data

In [ ]:
data_leaked

In [ ]:
#data_leaked_f = data_leaked.iloc[:7110]
data_leaked_f = data_leaked.iloc[7110:]
data_leaked_f.index = np.arange(len(data_leaked_f))
data_leaked_f

In [ ]:
#df.groupby(['Animal']).mean()
data_leaked_f.groupby(['NOx(GT)']).count()

In [ ]:
test

In [ ]:
leaked_co = np.array(data_leaked_f['CO(GT)'])
leaked_benzene = np.array(data_leaked_f['C6H6(GT)'])
leaked_nox = np.array(data_leaked_f['NOx(GT)'])

In [ ]:
for i in range(len(data_leaked_f)):
    if leaked_co[i] == -200:
        leaked_co[i] = co_pred[i]
    if leaked_benzene[i] == -200:
        leaked_benzene[i] = benzene_pred[i]
    if leaked_nox[i] == -200:
        leaked_nox[i] = nox_pred[i]

In [ ]:
output = pd.DataFrame({'date_time': pd.read_csv('/kaggle/input/tabular-playground-series-jul-2021/test.csv').date_time,
                       'target_carbon_monoxide': leaked_co, 
                       'target_benzene': leaked_benzene, 
                       'target_nitrogen_oxides': leaked_nox})

output.to_csv('submission.csv', index = False)
output

In [ ]:
output.describe()